In [ ]:
import numpy as np
import pandas as pd
import neo4j

def make_dict(df):
  sys_dict = {}
  for i in range(len(df)):
    # All names will be UPPERCASE to avoid matching errors
    std = df['stdname'][i].upper()
    sys = df['sysname'][i].upper()
    if std != "UNCHARACTERIZED":
      sys_dict[std] = sys
    else: 
      sys_dict[sys] = sys
  return sys_dict
     
def prepare():
    names_df = pd.read_csv('/project_data/gene_data_locus.csv')
    func_df = pd.read_csv('/project_data/gene_data_GO.csv')
    #print(func_df.head())
    #print(name_df.head())

    sys_dict = make_dict(names_df)
     
    # Remove extra information from each process
    func_df['go_MolFunc'] = [x.split(':')[0] for x in func_df['go_MolFunc']]
    func_df['go_CellComp'] = [x.split(':')[0] for x in func_df['go_CellComp']]
    func_df['go_BioProc'] = [x.split(':')[0] for x in func_df['go_BioProc']]

    # UPPPERCASE all names to avoid matching errors
    standardized = []
    for x in func_df['proteinname']:
      if x[-1] == 'p':
        standardized.append(x[0:-1].upper())
      else:
        standardized.append(x.upper())
    func_df['proteinname'] = standardized

    func_df['sysname'] = [sys_dict[std] for std in func_df['proteinname']]

    new_path = f'/project_data/truncated.csv'
    func_df.reset_index(drop=True, inplace=True)
    func_df.to_csv(new_path)
    print(func_df.drop(['description', 'go_BioProc', 'go_MolFunc'], axis=1).head(20))

prepare()

names_df = pd.read_csv('/project_data/gene_data_locus.csv')
#print(names_df.head())

seq_df = pd.read_csv('/project_data/gene_seq.csv')
seq_df.drop('promoter', axis=1, inplace=True)
#print(seq_df.head())

func_df = pd.read_csv('/project_data/gene_data_GO.csv')
#func_df.drop('go_CellComp', axis=1, inplace=True)
# simplify cell comp
func_df['go_CellComp'] = [x.split(':')[0] for x in func_df['go_CellComp']]

# 578 unique values
func_df['go_MolFunc'] = [x.split(':')[0] for x in func_df['go_MolFunc']]
#print(func_df['go_MolFunc'].value_counts().head(50))

func_df['go_BioProc'] = [x.split(':')[0] for x in func_df['go_BioProc']]
#print(func_df['go_BioProc'].nunique())
#print(func_df['go_BioProc'].value_counts().tail())

# Need to combine func_df and names_df, matching std/sysname on protein
#print(func_df)
#print(names_df)

# Remove all 'p's from proteinnames so we can match with other dataframe
cut = []
for x in func_df['proteinname']:
  if x[-1] == 'p':
    cut.append(x[0:-1].upper())
  else:
    cut.append(x.upper())
func_df['proteinname'] = cut
#print(func_df['proteinname'])

#func_df['proteinname'] = [x[0:-1] for x in func_df['proteinname'] if x[-1] == 'p']
#func_df.sort_values(by='proteinname', inplace=True)
#names_df.sort_values(by='stdname', inplace=True)
#func_df['sysname'] = [names_df_]
#func_df.merge(names_df['sysname'], how='left', left_on='proteinname', right_on='stdname')
print(func_df)


new_path = f'/project_data/truncated.csv'
func_df.to_csv(new_path)